<h1> Keiland Pullen DSC 478 - Programming Machine Learning Homework #4 </h1>

<h1>Problem 1.</h1>

<h2>Recommender System</h2>

In [307]:
from numpy import *
from numpy import linalg as la

import numpy as np
import pandas as pd

def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal
    
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    data=mat(dataMat)
    U,Sigma,VT = la.svd(data)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = data.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = data[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

# This function is not needed for Assignment 4, but may be useful for experimentation
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1] #find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

# This function performs evaluatoin on a single user based on the test_ratio
# For example, with test_ratio = 0.2, a randomly selected 20 percent of rated 
# items by the user are withheld and the rest are used to estimate the withheld ratings

def cross_validate_user(dataMat, user, test_ratio, estMethod=standEst, simMeas=pearsSim):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = int(test_ratio * len(rated_items_by_user))
    test_indices = np.random.randint(0, len(rated_items_by_user), test_size)
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # So that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)

    # Compute absolute error for user u over all test items
    for item in withheld_items:
        # Estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])

    # Now restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]

    # Return sum of absolute errors and the count of test cases for this user
    # Note that these will have to be accumulated for each user to compute MAE
    return error_u, count_u
    
#def test(dataMat, test_ratio, estMethod):
    # Write this function to iterate over all users and for each perform evaluation by calling
    # the above cross_validate_user function on each user. MAE will be the ratio of total error 
    # across all test cases to the total number of test cases, for all users
    #print ('Mean Absoloute Error for ',estMethod,' : ', MAE)

#def print_most_similar_jokes(dataMat, jokes, queryJoke, k, metric=pearsSim):
    # Write this function to find the k most similar jokes (based on user ratings) to a queryJoke
    # The queryJoke is a joke id as given in the 'jokes.csv' file (an corresponding to the a column in dataMat)
    # You must compare ratings for the queryJoke (the column in dataMat corresponding to the joke), to all
    # other joke rating vectors and return the top k. Note that this is the same as performing KNN on the 
    # columns of dataMat. The function must retrieve the text of the joke from 'jokes.csv' file and print both
    # the queryJoke text as well as the text of the returned jokes.
        
            
def load_jokes(file):
    jokes = genfromtxt(file, delimiter=',', dtype=str)
    jokes = np.array(jokes[:,1])
    return jokes

def get_joke_text(jokes, id):
    return jokes[id]

<h3>a.) Load in the joke ratings data and the joke text data into appropriate data structures.</h3>

In [308]:
jokeData = load_jokes('C:/Users/Home/Desktop/DePaul/Winter - DSC - 478 - Programming Machine Learning/Week 9/Homework-4/jokes/jokes.csv')

jokeRatings = np.genfromtxt('C:/Users/Home/Desktop/DePaul/Winter - DSC - 478 - Programming Machine Learning/Week 9/Homework-4/jokes/modified_jester_data.csv', delimiter=',')


In [309]:
jokeData.shape

(100,)

In [310]:
jokeRatings.shape

(1000, 100)

In [311]:
#jokeRatings
#jokeData

<h3>b.) Complete the definition for the function "test".... </h3>

In [312]:
def test(dataMat, test_ratio, estMethod):
    
    #print ('Inside of test function')
    #print ('The dataMat is', dataMat)
    #print ('The test ratio is', test_ratio)
    #print ('The estimation method is',estMethod)
    
    totalErr = 0
    totalCnt = 0
    
    meanAbsErr = 0
    
    for user in range(1, dataMat.shape[1]):
        data = cross_validate_user(dataMat, user, test_ratio, estMethod=standEst)
        totalErr = totalErr + data[0]
        totalCnt = totalCnt + data[1]
        
    meanAbsErr = totalErr/totalCnt
    
    
    print('The Mean Absolute Error for ', estMethod,' is ',meanAbsErr )
    
    #print('End of Test function.')

In [313]:
stand_Est = test(jokeRatings, 0.20, estMethod='standEst')

The Mean Absolute Error for  standEst  is  3.633388980526578


In [314]:
svd_Est = test(jokeRatings, 0.20, estMethod='svdEst')

The Mean Absolute Error for  svdEst  is  3.6833326477202095


<h3>c.) Write a new function "print_most_similar_jokes" which takes teh joke ratings data, ... </h3>

In [315]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

In [316]:
def print_most_similar_jokes(dataMat, jokes, queryJoke, k, metric=pearsSim):
    
    dataMat_T = dataMat.T
    user = dataMat_T.shape[0]
    joke = dataMat_T[queryJoke]
    
    # print ('jokes :', jokes)
    # print ('ratings :', dataMat)
    
    x = 0
    similar_tot = np.zeros((user, 1))
    similar_inx = np.zeros((user, 1))
    
    #print('Array of zeroes:', similar_tot)
    #print('Array of zeroes:', similar_inx)
    
    while x < user:
        similarity = metric(dataMat_T[x], joke)
        similar_tot[x] = similarity
        similar_inx[x] = x
        x = x + 1
    
    #print(similar_tot)
    #print(similar_inx)        
   
    #similar_inx = similar_inx.astype(int)
    
    AllJokes = np.concatenate( (similar_tot, similar_inx), axis = 1)
    #print(AllJokes)
    #print(type(AllJokes))
    
    print(" The Query joke is, ", jokes[queryJoke])
    print("\n")    
    
    similarJokes = []
    
    for i in AllJokes[:k]:
        #print(AllJokes)
        similarJokes.append(i)
        
    print(" The Simnilar jokes are: ")
    #print(similarJokes)
    
    z = 1
    for jk in similarJokes:
        #print(jokes[:])
        #print("jk = ",jk[1:])
        z = int(jk[1:])
        print(z)
        #print(type(jk))
        print(jokes[z])
    

In [317]:
print_most_similar_jokes(jokeRatings, jokeData, 9, 5,pearsSim)

 The Query joke is,  Two cannibals are eating a clown one turns to other and says: "Does this taste funny to you?


 The Simnilar jokes are: 
0
A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer's disease". The man replies "Well thank God I don't have cancer!"
1
This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That's an awfully big word for a ten year old."
2
Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.
3
Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.
4
Q. What's O. J. Simpson's Internet address? A.	Slash slash backslash slash slash escape.


In [318]:
print_most_similar_jokes(jokeRatings, jokeData, 6, 4, ecludSim)

 The Query joke is,  How many feminists does it take to screw in a light bulb?That's not funny.


 The Simnilar jokes are: 
0
A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer's disease". The man replies "Well thank God I don't have cancer!"
1
This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That's an awfully big word for a ten year old."
2
Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.
3
Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.


In [320]:
print_most_similar_jokes(jokeRatings, jokeData, 12, 4, metric = cosSim)

TypeError: only size-1 arrays can be converted to Python scalars

<h1>Problem 2.)</h1>

<h2>PCA for Reduced Dimensionality in Clusters</h2>

<h3>a.) Load in the image data matrix....</h3>

In [321]:
classes = pd.read_table('C:/Users/Home/Desktop/DePaul/Winter - DSC - 478 - Programming Machine Learning/Week 9/Homework-4/segmentation_data/segmentation_classes.txt', names=['term', 'class'], header=None)
features = pd.read_csv('C:/Users/Home/Desktop/DePaul/Winter - DSC - 478 - Programming Machine Learning/Week 9/Homework-4/segmentation_data/segmentation_data.txt', sep=',', header=None)
names = pd.read_table('C:/Users/Home/Desktop/DePaul/Winter - DSC - 478 - Programming Machine Learning/Week 9/Homework-4/segmentation_data/segmentation_names.txt', header=None)

In [328]:
#classes

In [329]:
from sklearn.preprocessing import MinMaxScaler

MinMax = MinMaxScaler()
feat_norm = MinMax.fit_transform(features)


<h3>b.) Next, Perform Kmeans clustering...</h3>

In [330]:
from sklearn.cluster import KMeans
from sklearn.metrics import homogeneity_completeness_v_measure

In [331]:
kmean_clf = KMeans(n_clusters = 7)
kmean_clf_fit = kmean_clf.fit(feat_norm)

kmean_pred = kmean_clf_fit.predict(feat_norm)


In [335]:
centers = kmean_clf_fit.cluster_centers_
#centers

centers = centers.T

In [336]:
pd.DataFrame(centers,columns=['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6', 'Cluster 7' ])

,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7
0,0.513994,0.770674,0.535099,0.256103,0.254169,0.750696,0.302506
1,0.808937,0.425215,0.150167,0.393468,0.459974,0.534564,0.530862
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.077441,0.013978,0.027778,0.074510,0.026256,0.040000,0.052260
4,0.005051,0.022581,0.001667,0.019118,0.013699,0.038462,0.046610
5,0.054474,0.040237,0.030228,0.077343,0.037274,0.114419,0.100817
6,0.001407,0.002989,0.000543,0.004100,0.002364,0.019301,0.009420
7,0.046335,0.023122,0.026766,0.060574,0.027874,0.109240,0.083972
8,0.001401,0.002088,0.000587,0.004967,0.002017,0.017999,0.011043
9,0.108790,0.041138,0.823246,0.148187,0.026013,0.300955,0.400608


In [337]:
score = homogeneity_completeness_v_measure(kmean_pred, classes['class'])

print ('Homogeneity score is ', score[0] )
print ('Completeness score is ', score[1] )

Homogeneity score is  0.6131870124853009
Completeness score is  0.6115021163370862


<h3>c.) Perform PCA on the normalized image data matrix...</h3>

In [338]:
from sklearn.decomposition import PCA

In [339]:
pca = PCA(n_components = 5)

pca_fit = pca.fit(feat_norm)

print('pca variance ration: ',pca.explained_variance_ratio_)
print('total explained variance: ', pca.explained_variance_ratio_.sum())

pca variance ration:  [0.60714234 0.13196979 0.10123773 0.04543539 0.03547361]
total explained variance:  0.9212588648109568


In [340]:
pca = PCA(n_components = 6)

pca_fit = pca.fit(feat_norm)

print('pca variance ration: ',pca.explained_variance_ratio_)
print('total explained variance: ', pca.explained_variance_ratio_.sum())

pca variance ration:  [0.60714234 0.13196979 0.10123773 0.04543539 0.03547361 0.01988035]
total explained variance:  0.9411392197960617


In [341]:
pca = PCA(n_components = 7)

pca_fit = pca.fit(feat_norm)

print('pca variance ration: ',pca.explained_variance_ratio_)
print('total explained variance: ', pca.explained_variance_ratio_.sum())

pca variance ration:  [0.60714234 0.13196979 0.10123773 0.04543539 0.03547361 0.01988035
 0.0189197 ]
total explained variance:  0.9600589227704962


In [342]:
pca = PCA(n_components = 7)
pca_fit = pca.fit(feat_norm)

transformed_pca_fit = pca.fit_transform(feat_norm)

<h3>d.) Perform Kmeans again, but this time on the lower dimensional transformed data</h3>

In [347]:
kmean_clf_2 = KMeans(n_clusters = 7)
kmean_clf_fit_2 = kmean_clf.fit(transformed_pca_fit)

kmean_pred_2 = kmean_clf_fit.predict(transformed_pca_fit)

In [348]:
centers_2 = kmean_clf_fit_2.cluster_centers_
#centers

In [349]:
pd.DataFrame(centers_2,columns=['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6', 'Cluster 7' ])

,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7
0,0.176404,0.043698,-0.265370,0.184124,0.027075,0.024341,0.003264
1,1.414527,0.087223,0.036765,-0.173195,-0.029922,-0.008973,-0.021573
2,-0.511579,-0.064908,-0.336145,-0.065365,0.078809,0.006240,-0.026261
3,-0.619268,0.640250,0.195829,-0.086856,-0.067760,0.008866,0.038372
4,-0.206229,-0.246205,0.152785,0.056522,0.130585,-0.005540,0.032849
5,-0.603705,-0.355503,0.109197,-0.129799,-0.130911,-0.021603,-0.043882
6,0.436890,-0.104936,0.164951,0.234379,-0.046159,-0.007512,0.015253


In [350]:
score = homogeneity_completeness_v_measure(kmean_pred_2, classes['class'])

print ('Homogeneity score is ', score[0] )
print ('Completeness score is ', score[1] )

Homogeneity score is  0.6107955063694607
Completeness score is  0.6091364049733291


<h3>e.) Discuss your oberservations based on the comparison of the two clustering results.</h3>

The clustering results appear to be very similar.  However, using the transformed data for dimensionality reduction with n = 7 (which captures 95% of the variance) features would be optimal instead of using all of the features, as this could improve performance time.